 #  **Design and Developement of Deep learning model for Rice Quality Analysis**

### **Importing necessary libraries**

In [ ]:
import numpy as np
import pandas as pd
from skimage import measure
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.utils import class_weight
import numpy as np
import matplotlib.pyplot as plt

### **Analysing single rice image and trying to resize it**

In [ ]:
img = cv2.imread('/kaggle/input/rice-image1600/rice_image16000/Basmati/basmati007.jpg')
plt.imshow(img)
plt.title("Original Image")
plt.axis('off')
plt.show()

In [ ]:
img.shape

In [ ]:
resized_image = cv2.resize(img, (700, 700))

plt.imshow(resized_image)
plt.title("resized image")
plt.axis('off')
plt.show()

In [ ]:
resized_image.shape

### **Implementing Function for rice feature Extraction**

In [ ]:
def calculate_features(image_path):
    
    #loading images
    image0 = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    #resizing images
    image = cv2.resize(image0, (1000, 1000))
    
    #Segmentation of image
    _, thresh = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((5, 5), np.uint8)
    morph_image = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(morph_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #Extracting the ROI
    largest_contour = max(contours, key=cv2.contourArea)

    #Calculating the area and perimeter from the extracted largest contour
    area = cv2.contourArea(largest_contour)
    perimeter = cv2.arcLength(largest_contour, True)

    #This is for checking the largest_contour is >5 or not, because in case of <5 it will not be able to fit in any type of ellipse
    if len(largest_contour) >= 5:
        
        #Trying to find the best fit ellipse
        ellipse = cv2.fitEllipse(largest_contour)
        
        #Extracting the major and minor axis length from best fitted ellipse
        minor_axis, major_axis = ellipse[1]
        
        #Calculating the aspect ratio
        aspect_ratio = major_axis / minor_axis
    else:
        
        #If largest_contour is <5 then major, minor and aspect ratio will be not a number(nan)
        major_axis = minor_axis = aspect_ratio = np.nan

    #Calculating the eccentricity of the ellipse
    if minor_axis < major_axis:
        eccentricity = np.sqrt(1 - (minor_axis / major_axis) ** 2)
    else:
        eccentricity = np.nan

    #Calculating the other morphological features of rice
    convex_hull = cv2.convexHull(largest_contour)
    convex_area = cv2.contourArea(convex_hull)

    equiv_diameter = np.sqrt(4 * area / np.pi)

    x, y, w, h = cv2.boundingRect(largest_contour)
    bounding_box_area = w * h

    extent = area / bounding_box_area

    solidity = area / convex_area

    roundness = (4 * area) / (np.pi * major_axis ** 2)

    compactness = (perimeter ** 2) / (4 * np.pi * area)

    #Defining four different shape factors
    shape_factor_1 = major_axis / area
    shape_factor_2 = minor_axis / area
    shape_factor_3 = area / ((major_axis / 2) * np.pi)
    shape_factor_4 = area / ((major_axis / 2) * (minor_axis / 2) * np.pi)

    #Create a dictionary with feature names and values
    path1 = os.path.dirname(image_path)
    features_dict = {
        "RiceName": os.path.basename(path1),
        "Area": area,
        "Perimeter": perimeter,
        "MajorAxisLength": major_axis,
        "MinorAxisLength": minor_axis,
        "AspectRatio": aspect_ratio,
        "Eccentricity": eccentricity,
        "ConvexArea": convex_area,
        "EquivalentDiameter": equiv_diameter,
        "Extent": extent,
        "Solidity": solidity,
        "Roundness": roundness,
        "Compactness": compactness,
        "ShapeFactor1": shape_factor_1,
        "ShapeFactor2": shape_factor_2,
        "ShapeFactor3": shape_factor_3,
        "ShapeFactor4": shape_factor_4,
    }

    return features_dict

### **Create an empty DataFrame to store the features**

In [ ]:
features_df = pd.DataFrame(columns=["RiceName", "Area", "Perimeter", "MajorAxisLength",
                                     "MinorAxisLength", "AspectRatio", "Eccentricity",
                                     "ConvexArea", "EquivalentDiameter", "Extent",
                                     "Solidity", "Roundness", "Compactness",
                                     "ShapeFactor1", "ShapeFactor2", "ShapeFactor3",
                                     "ShapeFactor4"])

### **Iterate through the images and calculate features**

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/rice-image1600'):
    for filename in filenames:
        image_path = os.path.join(dirname, filename)
        features_dict = calculate_features(image_path)
        features_row = pd.DataFrame([features_dict])
        
        features_df = pd.concat([features_df, features_row], ignore_index=True)

### **Save the DataFrame to a CSV file**

In [ ]:
features_df.to_csv('rice_features.csv', index=False)

### **Loading the dataset**

In [ ]:
data = pd.read_csv('/kaggle/input/rice-features/rice_features (2).csv')

In [ ]:
data.head()

### **Splitting dataset int X and Y**

In [ ]:
X = data.drop('RiceName', axis=1)
y = data['RiceName']

### **Initializing some empty list**

In [ ]:
f1_scores = []
recalls = []
precisions = []

### **Encoding Categorical Labels and Standardizing Features in Data Preprocessing**

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### **Building a Deep Neural Network with Dropout for Multiclass Classification**

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_scaled.shape[1]))
model.add(Dropout(0.5))  

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))

model.add(Dropout(0.5))


model.add(Dense(len(label_encoder.classes_), activation='softmax'))

### **Computing Class Weights for Handling Imbalanced Data in Classification**

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

### **Compiling a Neural Network Model for Multiclass Classification**


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### **Stratified K-Fold Cross-Validation for Model Evaluation**

In [ ]:
num_folds = 6
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

fold_accuracies = []     

### **Training Neural Network with Stratified K-Fold Cross-Validation**

In [ ]:
for i, (train_index, val_index) in enumerate(kf.split(X_scaled, y_encoded)):
    X_train_fold, X_val_fold = X_scaled[train_index], X_scaled[val_index]
    y_train_fold, y_val_fold = y_encoded[train_index], y_encoded[val_index]

    
    model.fit(X_train_fold, y_train_fold, epochs=120, batch_size=128, class_weight=class_weight_dict, verbose=0)

    
    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    fold_accuracies.append(val_accuracy)
    print(f'Fold {i + 1} - Validation Accuracy: {val_accuracy * 100:.3f}%')

### **Plotting Cross-Validation Accuracy of Neural Network Model**

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(1, num_folds + 1), [acc * 100 for acc in fold_accuracies], marker='o', linestyle='-')
plt.xlabel('Fold')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy of Each Fold')
plt.grid(True)
plt.xticks(range(1, num_folds + 1))
plt.show()